In [33]:
import nltk
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS,TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
def stumbleUpon():
    
    df_train=pd.read_csv('/content/train.tsv',delimiter='\t')

    df_test=pd.read_csv('/content/test.tsv',delimiter='\t')
    

    y_train = df_train.label.values
    
    urlid_test = df_test.urlid

    len_train = df_train.shape[0]

    train = df_train.boilerplate 
    test = df_test.boilerplate

    train = [i[9:-2] for i in train]
    test = [i[9:-2] for i in test]

    text = train + test

    tfidf_v = TfidfVectorizer(stop_words='english',min_df=3,strip_accents='unicode',token_pattern=r'\w{1,}',ngram_range=(1,2),use_idf=True, smooth_idf=True, sublinear_tf=True)
    tfidf_v.fit(text)
    X= tfidf_v.transform(text)

    # Seperating test and train file
    X_train= X[:len_train]
    X_test = X[len_train:]
    
    lr = LogisticRegression(class_weight = 'balanced')
    print('Cross Val Score: {:.2f}'.format(np.mean(cross_val_score(lr,X_train,y_train,cv=3,scoring='roc_auc'))))
    
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)
    pred = lr.predict(X_train)
    
    print(classification_report(y_train,pred))
    data = {'urlid':urlid_test,'label':y_pred}
    
    submit =pd.DataFrame(data)
    submit.to_csv('Submit.csv',index=False)
    
    
    
if __name__=="__main__":
    stumbleUpon()
    

Cross Val Score: 0.87
              precision    recall  f1-score   support

           0       0.83      0.93      0.88      3599
           1       0.92      0.82      0.87      3796

    accuracy                           0.87      7395
   macro avg       0.88      0.88      0.87      7395
weighted avg       0.88      0.87      0.87      7395

